# DATA PREPROCESSING


In [202]:
#Both Texts are premodified to have only relevant content
#non-ascii inverted commas converted to ascii types in text files
#regex removes non-relevant symbols and chapter numbers

import re
import io
import string

path = ""

with io.open(path+"speeches.txt","r",encoding="utf-8") as Speech:
    Text = re.sub("SPEECH [0-9]+","",Speech.read())
with io.open(path+"Jane_Austen.txt","r",encoding="utf-8") as JA:
        Text+= re.sub("Chapter [0-9]+","",JA.read())
Text = re.sub("[\"_,]","",Text)
Text = re.sub("\n"," ",Text)
Text = Text.replace(";","")
Text = re.sub("--","",Text)
Text = re.sub("(?:\.\.\.)(?=[A-Z])","",Text)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [203]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Classical Approach

In [0]:
from sklearn.model_selection import train_test_split

#Sentence tokenisation
from nltk.tokenize import sent_tokenize
sent = sent_tokenize(Text.lower())

#Processing sentences into list of words
for i in range(len(sent)):
  sent[i] = " ".join(re.findall("#?[A-Za-z]+[\.\'?!-]*[A-Za-z]*[\.\'?!-]*",sent[i]))
  sent[i] = "/ " + sent[i][:-1] + " \\"

#Train and Test data split
s_train,s_test = train_test_split(sent,test_size=0.2,random_state=1)
#Counting N-grams
VT = set()
UD = {} #uni
BD = {} #bi
TD = {} #tri
QD = {} #quad
TN=0
for stl in sent:
  st = stl.split()
  TN+=len(st)
  for j in range(len(st)):
    VT.add(st[j])
    if j+3<len(st):
      if st[j] not in UD:
        UD[st[j]] = 0
      UD[st[j]]+=1
      if st[j] + ' ' + st[j+1] not in BD:
        BD[st[j] + ' ' + st[j+1]] = 0
      BD[st[j] + ' ' + st[j+1]] += 1
      if st[j] + ' ' + st[j+1] + ' ' + st[j+2] not in TD:
        TD[st[j] + ' ' + st[j+1] + ' ' + st[j+2]] = 0
      TD[st[j] + ' ' + st[j+1] + ' ' + st[j+2]] += 1
      if st[j] + ' ' + st[j+1] + ' ' + st[j+2] + ' ' + st[j+3] not in QD:
        QD[st[j] + ' ' + st[j+1] + ' ' + st[j+2] + ' ' + st[j+3]] = 0
      QD[st[j] + ' ' + st[j+1] + ' ' + st[j+2] + ' ' + st[j+3]] += 1
    elif j+2<len(st):
      if st[j] not in UD:
        UD[st[j]] = 0
      UD[st[j]]+=1
      if st[j] + ' ' + st[j+1] not in BD:
        BD[st[j] + ' ' + st[j+1]] = 0
      BD[st[j] + ' ' + st[j+1]] += 1
      if st[j] + ' ' + st[j+1] + ' ' + st[j+2] not in TD:
        TD[st[j] + ' ' + st[j+1] + ' ' + st[j+2]] = 0
      TD[st[j] + ' ' + st[j+1] + ' ' + st[j+2]] += 1
    elif j+1<len(st):
      if st[j] not in UD:
        UD[st[j]] = 0
      UD[st[j]]+=1
      if st[j] + ' ' + st[j+1] not in BD:
        BD[st[j] + ' ' + st[j+1]] = 0
      BD[st[j] + ' ' + st[j+1]] += 1
    else:
      if st[j] not in UD:
        UD[st[j]] = 0
      UD[st[j]]+=1
V = len(VT)
VT = sorted(list(VT-{"/"}))

### Random Sentence Generator

In [0]:
import numpy as np

def nW(N,W,P):
  return W[np.argmax(np.random.multinomial(10,P))]

def genWPlist(given,UD,BD,TD,QD):
  n = len(given) + 1
  W = []
  P = []
  if n==2:
    for bgrm in BD.keys():
      Li = bgrm.split()
      if Li[:-1] == given:
        W.append(Li[1])  #appending the next word candidate
        P.append(BD[bgrm]/UD[Li[0]]) #appending the next word candidate's MLE 
  if n==3:
    for tgrm in TD.keys():
      Li = tgrm.split()
      if Li[:-1] == given:
        W.append(Li[2]) #appending the next word candidate
        P.append(TD[tgrm]/BD[Li[0]+' '+Li[1]]) #appending the next word candidate's MLE 
  if n==4:
    for qgrm in QD.keys():
      Li = qgrm.split()
      if Li[:-1] == given:
        W.append(Li[3]) #appending the next word candidate
        P.append(QD[qgrm]/TD[Li[0]+' '+Li[1]+' '+Li[2]]) #appending the next word candidate's MLE 
  return W,P

def Generator(N,UD,BD,TD,QD,TN,VT):
  randtext = ""
  for i in range(5):
    rsent = ["/"]
    if N==1:
      W = VT[:]  #next word candidates
      P = list(map(lambda x:UD[x]/TN,W[:])) #MLE of the next words in unigram model
      while True:
        rsent.append(W[np.argmax(np.random.multinomial(10,P))]) #appending the next word
        if rsent == ["/","\\"]:
          rsent = ["/"]
          continue
        if rsent[-1]=="\\":
          randtext += " ".join(rsent[1:-1]) + "."
          break  #stopping if next word was end of sentence
    else:
      while True:
        given = rsent[max(0,len(rsent)-N+1):len(rsent)]
        W,P = genWPlist(given,UD,BD,TD,QD) #generating next word candidates and MLE's of them
        rsent.append(W[np.argmax(np.random.multinomial(10,P))]) #appending the next word
        if rsent[-1]=="\\":
          if len(rsent)<N:
            continue
          randtext += " ".join(rsent[1:-1]) + "."
          break  #stopping if next word was end of sentence
  return randtext

In [206]:
#Unigram text generator
print(Generator(1,UD,BD,TD,QD,TN,VT))

agitation always the zones at.a and absurdity body's.conversation it greatly believing a.a at all but any followed are almost and as.zones and aware to and from but had had around of.


In [207]:
#Bigram text generator
print(Generator(2,UD,BD,TD,QD,TN,VT))

but he was with her husband.i shall not be.i am not been a lot of the first day and by the way.she was the country.i think of the room.


In [208]:
#Trigram text generator
print(Generator(3,UD,BD,TD,QD,TN,VT))

we have a problem that we shall be a very pretty shrubbery.but you know i'm going to happen.but i can only do so wish that he was a great job.they are to part with them.we're going to be very agreeable he had a far more than once.


In [209]:
#Quadgram text generator, Note: Sometimes throws error due to non-occurence of some quadgram with a certain word.
print(Generator(4,UD,BD,TD,QD,TN,VT))

he had been very little within that sum.he was a great deal to say in reply.that's what going to happen.and the reason i can say that i am not going to happen.and i have a great success you have to hit back hard and you can't solve the problem.


#### The sentences get more readable at higher n gram models.

### Perplexity Calculations

In [0]:
#Computing Perplexity on Test with add-1 smoothing
from math import log,exp

def cnt(st,N,UD,BD,TD,QD): #returns count of n grams
  if N==1:
    if st not in UD:
      return 0
    return UD[st]
  elif N==2:
    if st not in BD:
      return 0
    return BD[st]
  elif N==3:
    if st not in TD:
      return 0
    return TD[st]
  elif N==4:
    if st not in QD:
      return 0
    return QD[st]

lpu = 0
lpb = 0
lpt = 0
lpq = 0

t = 0
#computing log sum of MLE's with smoothing

for ts in s_test:
  lts = ts.split()
  t+=1
  for j in range(1,len(lts)-1):
    if j-3>=0:
      lpu+=log( ( cnt(lts[j],1,UD,BD,TD,QD) + 1 )/(TN + V) )
      lpb+=log( ( cnt(lts[j-1]+' '+lts[j],2,UD,BD,TD,QD) + 1 )/( cnt(lts[j-1],1,UD,BD,TD,QD) + V ) )
      lpt+=log( ( cnt(lts[j-2]+' '+lts[j-1]+' '+lts[j],3,UD,BD,TD,QD) + 1)/( cnt(lts[j-2]+' '+lts[j-1],2,UD,BD,TD,QD) + V ) )
      lpq+=log( ( cnt(lts[j-3]+' '+lts[j-2]+' '+lts[j-1]+' '+lts[j],4,UD,BD,TD,QD) + 1 )/( cnt(lts[j-3]+' '+lts[j-2]+' '+lts[j-1],3,UD,BD,TD,QD) + V ) )
    elif j-2>=0 and j-3<0: 
      lpu+=log( ( cnt(lts[j],1,UD,BD,TD,QD) + 1 )/( TN + V ) )
      lpb+=log( ( cnt(lts[j-1]+' '+lts[j],2,UD,BD,TD,QD) + 1 )/( cnt(lts[j-1],1,UD,BD,TD,QD) + V ) )
      lpt+=log( ( cnt(lts[j-2]+' '+lts[j-1]+' '+lts[j],3,UD,BD,TD,QD) + 1)/( cnt(lts[j-2]+' '+lts[j-1],2,UD,BD,TD,QD) + V ) )
      lpq+=log( ( cnt(lts[j-2]+' '+lts[j-1]+' '+lts[j],3,UD,BD,TD,QD) + 1)/( cnt(lts[j-2]+' '+lts[j-1],2,UD,BD,TD,QD) + V ) )
    elif j-1>=0 and j-2<0: 
      lpu+=log( ( cnt(lts[j],1,UD,BD,TD,QD) + 1 )/( TN + V ) )
      lpb+=log( ( cnt(lts[j-1]+' '+lts[j],2,UD,BD,TD,QD) + 1 )/( cnt(lts[j-1],1,UD,BD,TD,QD) + V ) )
      lpt+=log( ( cnt(lts[j-1]+' '+lts[j],2,UD,BD,TD,QD) + 1 )/( cnt(lts[j-1],1,UD,BD,TD,QD) + V ) )
      lpq+=log( ( cnt(lts[j-1]+' '+lts[j],2,UD,BD,TD,QD) + 1 )/( cnt(lts[j-1],1,UD,BD,TD,QD) + V ) )

In [211]:
print("Unigram Perplexity:",exp(-lpu/t))
print("Bigram Perplexity:",exp(-lpb/t))
print("Trigram Perplexity:",exp(-lpt/t))
print("Quadgram Perplexity:",exp(-lpq/t))

Unigram Perplexity: 7.364168684167195e+51
Bigram Perplexity: 1.0445888926731425e+55
Trigram Perplexity: 1.095581907081848e+66
Quadgram Perplexity: 1.0875556596948705e+69
